In [1]:
import matplotlib.pyplot as plt
from utils import NuScenesDataSet
from models import ultra_fast_bev, ultra_fast_bev_nmsbf
import numpy as np
import tensorflow as tf 

dataroot='../../../../datasets/nuscenes/'
nuscd = NuScenesDataSet(dataroot)

2024-02-16 18:08:56.392479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 18:08:56.392547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 18:08:56.393798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 18:08:56.404188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-16 18:08:57.060800: W tensorflow/compiler/tf2

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 29.806 seconds.
Reverse indexing ...
Done reverse indexing in 5.7 seconds.


create data pipeline 

In [3]:
batch_size = 5
train_camera_files_paths, val_camera_files_paths = nuscd.get_train_val_cameras_file_paths()

file_path_lidar_train = '../../../../datasets/lidar_voxels_train/'
file_path_lidar_val = '../../../../datasets/lidar_voxels_val/'
train_lidar_files_paths, val_lidar_files_paths = nuscd.get_train_val_file_paths(file_path_lidar_train, file_path_lidar_val)

file_path_bev_mask_train = '../../../../datasets/nusc_bev_mask_train/'
file_path_bev_mask_val = '../../../../datasets/nusc_bev_mask_val/'
train__bev_mask_files_paths, val__bev_mask_files_paths = nuscd.get_train_val_file_paths(file_path_bev_mask_train, file_path_bev_mask_val)

train_camera_files_paths = np.array(train_camera_files_paths)
val_camera_files_paths = np.array(val_camera_files_paths)
train_lidar_files_paths = np.array(train_lidar_files_paths)[..., np.newaxis]
val_lidar_files_paths = np.array(val_lidar_files_paths)[..., np.newaxis]
train__bev_mask_files_paths = np.array(train__bev_mask_files_paths)[..., np.newaxis]
val__bev_mask_files_paths = np.array(val__bev_mask_files_paths)[..., np.newaxis]

train_files_paths = np.concatenate((train_camera_files_paths, train_lidar_files_paths, train__bev_mask_files_paths), axis=-1)
val_files_paths = np.concatenate((val_camera_files_paths, val_lidar_files_paths, val__bev_mask_files_paths), axis=-1)


def get_image(image_path):
    image_shape = (448, 800)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_shape)
    return image

def parse_samples(file_paths):
    image0 = get_image(file_paths[0])
    image1 = get_image(file_paths[1])
    image2 = get_image(file_paths[2])
    image3 = get_image(file_paths[3])
    image4 = get_image(file_paths[4])
    image5 = get_image(file_paths[5])

    lidar = nuscd.read_binary_file(file_paths[6], dtype=tf.float32, shape=(200,200,4))
    bev_mask = nuscd.read_binary_file(file_paths[7], dtype=tf.float32, shape=(200,200,3))
    
    return ((image0, image1, image2, image3, image4, image5, lidar), bev_mask)

train_dataset = tf.data.Dataset.from_tensor_slices(train_files_paths)
val_dataset = tf.data.Dataset.from_tensor_slices(val_files_paths)

train_dataset = train_dataset.shuffle(buffer_size=train_dataset.cardinality(), reshuffle_each_iteration=True).map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(1)

val_dataset = val_dataset.map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset= val_dataset.batch(batch_size).prefetch(1)

print(train_files_paths.shape[0], val_files_paths.shape[0])
print(train_dataset.cardinality().numpy(), val_dataset.cardinality().numpy())

28130 6019
5626 1204


# test datapipeline 

In [ ]:
# scale = 1/255.0
# for x ,y in train_dataset:
#     top_img = tf.concat((x[1],x[0],x[2]), axis=2)
#     bottom_img = tf.concat((x[5],x[3],x[4]), axis=2)
#     img = tf.concat((top_img, bottom_img), axis=1)[0].numpy()*scale
#     lidar = x[6][0].numpy()
#     bev_mask = y[0].numpy()
#     bev_mask = nuscd.add_ego_vehicle_to_mask(bev_mask)
#     bev_mask = nuscd.from_mask_to_rgb(bev_mask)
#     break
# stop = 1

# plt.figure(figsize=(20,20))
# plt.subplot(2,2,(1,2))
# plt.axis(False)
# plt.imshow(img)

# plt.subplot(2,2,3)
# plt.axis(False)
# plt.imshow(bev_mask)

# plt.subplot(2,2,4)
# plt.axis(False)
# plt.imshow((np.sum(lidar, axis=-1)>=1)*1.0, cmap='gray')

# plt.subplots_adjust(wspace=0.0, hspace=0.0) 
# plt.show()


# Custom CallBack

In [6]:
class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        self.best_metric = 0

    def my_optimizer(self, epoch):
        lr = {5: 1e-4, 10:3e-5, 15:1e-5}
        if epoch in lr.keys():
            print(f'\n set learning rate to {lr[epoch]}')
            tf.keras.backend.set_value(self.model.optimizer.lr, lr[epoch])
            
    def on_epoch_end(self, epoch, logs=None):
        self.my_optimizer(epoch)
        if logs.get("val_Mean_IOU")>self.best_metric:
            self.best_metric = logs.get("val_Mean_IOU")
            self.model.save_weights(f'./model_weights/{self.model_name}.h5')
            print(f'\n {self.model_name} saved best weights')
    
        

# Train model 

## Train with multy scale bev features 

In [ ]:
tf.config.optimizer.set_jit(True) #Enable XLA
tf.keras.mixed_precision.set_global_policy('mixed_float16')

cameras_LUTs1 = nuscd.create_images_LUTs(image_size=(56,100))
cameras_LUTs2 = nuscd.create_images_LUTs(image_size=(112,200))
cameras_LUTs3 = nuscd.create_images_LUTs(image_size=(224,400))
cameras_LUTs = [cameras_LUTs1, cameras_LUTs1, cameras_LUTs2, cameras_LUTs3] 

#model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'GroupNormalization', './model_weights/image_encoder_effnetb1_groupnorm_bs20_imagenet.h5')
# log_dir = f"logs/ufb_groupnorm_bs2"

model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5')
# log_dir = f"logs/ufb_batchnorm_bs2"
log_dir = f"logs/ufb_batchnorm_bs4" #todo

# model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5', 'C')
# log_dir = f"logs/ufb_batchnorm_bs4_nl" 

loss = tf.keras.losses.CategoricalFocalCrossentropy()
optimizer = tf.keras.optimizers.AdamW(3e-4, weight_decay=1e-2)
metrics = [
    tf.keras.metrics.MeanIoU(num_classes=3, name = "Mean_IOU", sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "road", target_class_ids = [0], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "car", target_class_ids = [1], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "nothing", target_class_ids = [2], sparse_y_true= False, sparse_y_pred=False)
    ]

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir), MyCallback(log_dir.split('/')[-1])]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset, validation_data=val_dataset, epochs=16, callbacks=callbacks)

## Train with no multy scale bev features 

In [ ]:
tf.config.optimizer.set_jit(True) #Enable XLA
tf.keras.mixed_precision.set_global_policy('mixed_float16')

cameras_LUTs1 = nuscd.create_images_LUTs(image_size=(56,100))
cameras_LUTs = [cameras_LUTs1] 

# model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'GroupNormalization', './model_weights/image_encoder_effnetb1_groupnorm_bs20_imagenet.h5')
# log_dir = f"logs/ufb_groupnorm_bs2_nmsbf"

model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5')
# log_dir = f"logs/ufb_batchnorm_bs2_nmsbf"
# log_dir = f"logs/ufb_batchnorm_bs4_nmsbf"

model.load_weights('./model_weights/ufb_batchnorm_bs4_nmsbf.h5')
log_dir = f"logs/ufb_batchnorm_bs5_nmsbf" #trained using the weights of ufb_batchnorm_bs4_nmsbf

# model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5', 'C')
# log_dir = f"logs/ufb_batchnorm_bs4_nmsbf_nl" 

loss = tf.keras.losses.CategoricalFocalCrossentropy()
optimizer = tf.keras.optimizers.AdamW(3e-4, weight_decay=1e-2)
metrics = [
    tf.keras.metrics.MeanIoU(num_classes=3, name = "Mean_IOU", sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "road", target_class_ids = [0], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "car", target_class_ids = [1], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "nothing", target_class_ids = [2], sparse_y_true= False, sparse_y_pred=False)
    ]

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir), MyCallback(log_dir.split('/')[-1])]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_dataset, validation_data=val_dataset, epochs=16, callbacks=callbacks)

In [8]:
log_dir = f"logs/ufb_batchnorm_bs5_nmsbf_run2" #trained using the weights of ufb_batchnorm_bs4_nmsbf
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir), MyCallback(log_dir.split('/')[-1])]
tf.keras.backend.set_value(model.optimizer.lr, 1e-4)
model.fit(train_dataset, validation_data=val_dataset, epochs=24, callbacks=callbacks)

Epoch 1/24
5626/5626 [==============================] - ETA: 0s - loss: 0.0034 - Mean_IOU: 0.5227 - road: 0.4552 - car: 0.6201 - nothing: 0.4928
 ufb_batchnorm_bs5_nmsbf_run2 saved best weights
5626/5626 [==============================] - 1939s 344ms/step - loss: 0.0034 - Mean_IOU: 0.5227 - road: 0.4552 - car: 0.6201 - nothing: 0.4928 - val_loss: 0.0211 - val_Mean_IOU: 0.5766 - val_road: 0.5995 - val_car: 0.5217 - val_nothing: 0.6086
Epoch 2/24
5626/5626 [==============================] - 1930s 343ms/step - loss: 0.0034 - Mean_IOU: 0.5242 - road: 0.4566 - car: 0.6217 - nothing: 0.4944 - val_loss: 0.0227 - val_Mean_IOU: 0.5749 - val_road: 0.5954 - val_car: 0.5255 - val_nothing: 0.6039
Epoch 3/24
5626/5626 [==============================] - ETA: 0s - loss: 0.0033 - Mean_IOU: 0.5269 - road: 0.4595 - car: 0.6240 - nothing: 0.4973
 ufb_batchnorm_bs5_nmsbf_run2 saved best weights
5626/5626 [==============================] - 1941s 345ms/step - loss: 0.0033 - Mean_IOU: 0.5269 - road: 0.4595 - 

KeyboardInterrupt: 

# Visualize results

In [ ]:
# from matplotlib import pyplot as plt
# from IPython.display import clear_output
# import cv2 
# import keyboard
# import os 

# stop = 1
# key = ''
# sample = 0
# scale = 1/255.0
# y_pred_fig = None
# for x, y in val_dataset.skip(1400).unbatch().batch(1):
#     y_pred = model(x).numpy().reshape((200,200,3))
#     y_pred = tf.one_hot(np.argmax(y_pred, axis=-1),depth=3).numpy()
#     y_pred = nuscd.add_ego_vehicle_to_mask(y_pred, distance_around_ego=(100,100))
#     y_pred = nuscd.from_mask_to_rgb(y_pred)

#     y = nuscd.add_ego_vehicle_to_mask(y.numpy().reshape((200,200,3)), distance_around_ego=(100,100))
#     y = nuscd.from_mask_to_rgb(y)

#     lidar = (np.sum(x[6].numpy()[0, :, :, :], axis=-1)>=1)[..., np.newaxis]
#     lidar = np.concatenate((lidar,lidar,lidar),axis=-1)

#     y =  np.clip(y + lidar *0.4, 0, 1)

#     top_img = tf.concat((x[1],x[0],x[2]), axis=2)
#     bottom_img = tf.concat((x[5],x[3],x[4]), axis=2)
#     img = tf.concat((top_img, bottom_img), axis=1)[0].numpy()*scale
    
#     bev = np.concatenate((y, np.ones((y.shape[0],1,3), dtype=np.float32), y_pred), axis=1)
#     bev = cv2.resize(bev, (img.shape[1], int(bev.shape[0]*img.shape[1]/bev.shape[1]) ), cv2.INTER_LINEAR)

#     plot = np.concatenate((img, np.ones((6, img.shape[1],3)),bev), axis=0)

#     cv2.imshow('BEV',np.flip(plot, axis=-1))
    
#     if key == ord('s') or stop ==1:
#         key = cv2.waitKey(0) & 0xFF
#         stop = 1

#     if key != ord('s') or stop==0:
#         key = cv2.waitKey(1) & 0xFF
#         stop = 0

#     if key == ord('q'):
#         break

#     clear_output()
#     print(f'sample {sample}')
#     sample+=1
# cv2.destroyAllWindows()
